In [1]:
import numpy as np
import pandas as pd
import folium 
import matplotlib.pyplot as plt
from urllib.request import urlopen
import geocoder

# Scraping and cleaning data

Reading the data into the dataframe df. 

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, match='North York')

Setting up the column names properly, since it got read as a row

In [3]:
df = df[0].drop(0)
df.columns = ['Postal Code', 'Borough', 'Neighborhood']
df.reset_index(drop=True, inplace=True)

Dropping all the rows that have unassigned boroughs

In [4]:
df = df[df.Borough != 'Not assigned']

Joining the neighborhood names in the dataframe Toronto

In [5]:
postal_codes = list(set(df['Postal Code']))
Toronto = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood'])
for i,p in enumerate(postal_codes):
    temp_df = df[df['Postal Code']==p]
    neighb = ', '.join(list(temp_df['Neighborhood']))
    Toronto.loc[i] = [temp_df.iloc[0]['Postal Code'], temp_df.iloc[0]['Borough'], neighb]

Fill the neighborhood value with borough value where missing

In [6]:
Toronto['Neighborhood'].replace('Not assigned', np.nan, inplace=True)
Toronto['Neighborhood'].fillna(Toronto['Borough'], inplace=True)

In [7]:
Toronto.head()

,Postal Code,Borough,Neighborhood
0,M9A,Etobicoke,Islington Avenue
1,M4K,East Toronto,"The Danforth West, Riverdale"
2,M1B,Scarborough,"Rouge, Malvern"
3,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
4,M6J,West Toronto,"Little Portugal, Trinity"


In [8]:
Toronto.shape

(103, 3)

# Getting longitute and latitude

Sorting the 2 datasets makes the postal codes match up. so I can add the latitude and longitude values easily

In [9]:
coord = pd.read_csv('Geospatial_Coordinates.csv')

In [10]:
Toronto.sort_values(by='Postal Code', axis=0, inplace=True)

In [11]:
coord.sort_values(by='Postal Code', axis=0, inplace=True)

In [12]:
Toronto['Latitude']=coord['Latitude']
Toronto['Longitude']=coord['Longitude']

In [13]:
Toronto.reset_index(drop=True, inplace=True)

In [14]:
Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.763573,-79.188711
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.688905,-79.554724
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.648429,-79.382280
3,M1G,Scarborough,Woburn,43.716316,-79.239476
4,M1H,Scarborough,Cedarbrae,43.836125,-79.205636


In [15]:
Toronto.shape

(103, 5)

# Segmenting and clustering
### I'm mostly following the lab

Getting toronto location

In [16]:
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode("Toronto, CA")
latitude = location.latitude
longitude = location.longitude

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
CLIENT_ID = 'MDDGYA2LRHJSANOSNX4ELLUSBDJY3KQ4BMMWAQ2DII5TZXF1' # your Foursquare ID
CLIENT_SECRET = 'KKI202QA0UJYF3VETJTV4SJMGQAPNWGIPP5IRTPWZNEMPJ4V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

In [22]:
Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [23]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2237, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
1,"Rouge, Malvern",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Rouge, Malvern",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Rouge, Malvern",43.763573,-79.188711,chatr Mobile,43.766140,-79.191630,Electronics Store
4,"Rouge, Malvern",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [24]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

Toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [26]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Pizza Place,Fast Food Restaurant,Intersection,Gastropub,Athletics & Sports,Bank,Gym / Fitness Center,Pharmacy,Event Space,Ethiopian Restaurant
1,Agincourt,Park,Japanese Restaurant,Bakery,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,River,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Breakfast Spot,Coffee Shop,Café,Burrito Place,Caribbean Restaurant,Restaurant,Bar,Italian Restaurant,Bakery,Intersection
4,"Alderwood, Long Branch",Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant


In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged['Cluster Labels'].replace('Cluster Labels', np.nan)
Toronto_merged['Cluster Labels'].fillna(0, inplace=True)

Toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Intersection,Mexican Restaurant,Drugstore,Dive Bar,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.688905,-79.554724,2.0,Pizza Place,Park,Mobile Phone Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.648429,-79.382280,0.0,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Gastropub,Seafood Restaurant,American Restaurant,Asian Restaurant,Deli / Bodega
3,M1G,Scarborough,Woburn,43.716316,-79.239476,0.0,Motel,American Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.836125,-79.205636,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# I chose the number of clusters experimentally 
# Cluster 1 

In [30]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Intersection,Mexican Restaurant,Drugstore,Dive Bar,Dog Run
2,Scarborough,0.0,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Gastropub,Seafood Restaurant,American Restaurant,Asian Restaurant,Deli / Bodega
3,Scarborough,0.0,Motel,American Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,Scarborough,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Scarborough,0.0,Home Service,Baseball Field,Food Truck,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
6,Scarborough,0.0,Bank,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Dive Bar
7,Scarborough,0.0,Italian Restaurant,Sushi Restaurant,Coffee Shop,Liquor Store,Pizza Place,Fast Food Restaurant,Sandwich Place,Butcher,Café,Juice Bar
8,Scarborough,0.0,Park,Dance Studio,Skating Rink,Pharmacy,Video Store,Cosmetics Shop,Beer Store,Athletics & Sports,Bus Stop,Curling Ice
10,Scarborough,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Food Court,Japanese Restaurant,Bakery,Electronics Store,Asian Restaurant,Tea Room,Boutique
11,Scarborough,0.0,Gym,Beer Store,Coffee Shop,Discount Store,Asian Restaurant,Sandwich Place,Bike Shop,Sporting Goods Shop,Restaurant,Supermarket


# Cluster 2

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,1.0,Playground,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
100,Etobicoke,1.0,Convenience Store,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


# CLUSTER 3

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,Pizza Place,Park,Mobile Phone Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,Scarborough,2.0,Park,Convenience Store,Bank,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
12,Scarborough,2.0,Park,Japanese Restaurant,Bakery,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
14,Scarborough,2.0,Park,River,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
19,North York,2.0,Park,Airport,Other Repair Shop,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
48,Central Toronto,2.0,Park,Women's Store,Market,Fast Food Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
57,Downtown Toronto,2.0,Construction & Landscaping,Basketball Court,Park,Bakery,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
60,Downtown Toronto,2.0,Park,Playground,Building,Trail,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
71,North York,2.0,Park,Coffee Shop,Pizza Place,Convenience Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
76,West Toronto,2.0,Park,Swim School,Bus Line,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
